In [1]:
import pandas as pd
import sqlite3
import requests
import os
import pandas as pd
import sqlite3
import json


# Base URL for the raw files in the GitHub repo
base_url = "https://raw.githubusercontent.com/BigRoddy/CMIN-Dataset/main/CMIN-US/price/raw/"

# List of chosen stock CSV filenames
csv_files = ['AAPL.csv', 'AMZN.csv', 'BAC.csv', 'BA.csv', 'CVX.csv', 'CAT.csv', 'DIS.csv', 'JNJ.csv', 
             'MCD.csv', 'MSFT.csv', 'NKE.csv', 'PEP.csv', 'TSLA.csv', 'TGT.csv', 'XOM.csv', 'UNH.csv',
             'NEE.csv', 'RIO.csv', 'HD.csv', 'PG.csv']

# Connect to the SQLite database
conn = sqlite3.connect('cmin-us.db')

# Create a new table that will hold all the data from the 20 CSV files
for file in csv_files:
    # Build the full URL to the CSV file
    url = base_url + file
    
    # Download the CSV file
    response = requests.get(url)
    
    # Save the file locally
    with open(file, 'wb') as f:
        f.write(response.content)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file)
    
    # Add a column for the stock ticker (from the filename)
    df['Stock'] = file.split('.')[0]
    
    # Append the data to a single table in SQLite
    df.to_sql('prices', conn, if_exists='append', index=False)

# Close the SQLite connection
conn.close()


In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('cmin-us.db')

# Query to fetch the first 10 rows from the 'prices' table
query = "SELECT * FROM prices LIMIT 10"

# Load the query result into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()

         Date       Open       High        Low      Close  Adj Close  \
0  2018-01-02  42.540001  43.075001  42.314999  43.064999  41.188164   
1  2018-01-03  43.132500  43.637501  42.990002  43.057499  41.180992   
2  2018-01-04  43.134998  43.367500  43.020000  43.257500  41.372272   
3  2018-01-05  43.360001  43.842499  43.262501  43.750000  41.843311   
4  2018-01-08  43.587502  43.902500  43.482498  43.587502  41.687901   
5  2018-01-09  43.637501  43.764999  43.352501  43.582500  41.683121   
6  2018-01-10  43.290001  43.575001  43.250000  43.572498  41.673542   
7  2018-01-11  43.647499  43.872501  43.622501  43.820000  41.910255   
8  2018-01-12  44.044998  44.340000  43.912498  44.272499  42.343048   
9  2018-01-16  44.474998  44.847500  44.035000  44.047501  42.127838   

      Volume Stock  
0  102223600  AAPL  
1  118071600  AAPL  
2   89738400  AAPL  
3   94640000  AAPL  
4   82271200  AAPL  
5   86336000  AAPL  
6   95839600  AAPL  
7   74670800  AAPL  
8  101672400  AAPL

In [3]:
# List of chosen stock tickers
stocks = ['AAPL', 'AMZN', 'BAC', 'BA', 'CVX', 'CAT', 'DIS', 'JNJ', 
          'MCD', 'MSFT', 'NKE', 'PEP', 'TSLA', 'TGT', 'XOM', 'UNH',
          'NEE', 'RIO', 'HD', 'PG']

# Base URL for the preprocessed news files in the GitHub repo
news_base_url = "https://api.github.com/repos/BigRoddy/CMIN-Dataset/contents/CMIN-US/news/preprocessed/"

# Function to retrieve file list for a given stock from GitHub API
def get_available_files(stock):
    url = news_base_url + stock
    response = requests.get(url)
    if response.status_code == 200:
        file_data = response.json()
        # Extract file names (which represent dates)
        available_files = [file['name'] for file in file_data if file['type'] == 'file']
        return available_files
    else:
        print(f"Failed to retrieve file list for {stock}: {response.status_code}")
        return []

# Connect to the existing SQLite database
conn = sqlite3.connect('cmin-us.db')

# Loop through each stock
for stock in stocks:
    # Get the available file names (dates) for this stock
    available_files = get_available_files(stock)
    
    for file_name in available_files:
        # Build the raw content URL
        file_url = f"https://raw.githubusercontent.com/BigRoddy/CMIN-Dataset/main/CMIN-US/news/preprocessed/{stock}/{file_name}"
        
        # Download and read the file content
        response = requests.get(file_url)
        
        if response.status_code == 200:
            # Each line in the response is a JSON-like object
            lines = response.text.splitlines()
            
            # Parse each line and create a list of records
            news_data = []
            for line in lines:
                news_item = json.loads(line)  # The lines are JSON-like
                news_data.append({
                    'text': ' '.join(news_item['text']),  # Combine text into one string
                    'created_at': news_item['created_at'],
                    'date': file_name.replace('.json', ''),  # Use date from the file name
                    'stock': stock  # Stock ticker from folder name
                })
            
            # Convert the list of records into a DataFrame
            df = pd.DataFrame(news_data)
            
            # Append the data to the SQLite table ('news' table)
            df.to_sql('news_data', conn, if_exists='append', index=False)
        else:
            print(f"Failed to retrieve data for {stock} on {file_name}: {response.status_code}")

# Close the SQLite connection
conn.close()

In [4]:
# Connect to the SQLite database
conn = sqlite3.connect('cmin-us.db')

# Query to fetch the first 10 rows from the 'prices' table
query = "SELECT * FROM news_data LIMIT 10"

# Load the query result into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()

                                                text           created_at  \
0  Forget Your iPhone X, Ignore The Samsung And P...  2018-01-01 23:28:00   
1                  Apple's Executive Cash Bonus Plan  2018-01-01 22:19:00   
2     2 Warren Buffett Stocks to Consider Buying Now  2018-01-01 21:08:20   
3     2 Warren Buffett Stocks to Consider Buying Now  2018-01-01 20:02:00   
4   Bitcoin or Stocks? Here’s the One to Buy in 2018  2018-01-01 16:06:40   
5  Cramer reflects on how Trump's actions are fue...  2018-01-02 23:34:00   
6  Tech Leads Market Advance To New Highs; These ...  2018-01-02 23:08:52   
7  Tuesday Apple Rumors: Battery Replacement is A...  2018-01-02 23:07:18   
8  Here&apos;s Why Optical Networking Stocks Fell...  2018-01-02 23:00:00   
9             Why Apple Could Be a Big Buyer in 2018  2018-01-02 22:32:24   

         date stock  
0  2018-01-01  AAPL  
1  2018-01-01  AAPL  
2  2018-01-01  AAPL  
3  2018-01-01  AAPL  
4  2018-01-01  AAPL  
5  2018-01-02  AAPL 